<a href="https://colab.research.google.com/github/IlyaZutler/Bus_lanes/blob/main/DM%20_%20LinesGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
import folium
from geopy.distance import geodesic

In [71]:
# 'trips.xlsx' is uploaded to Colab environment
df_trips = pd.read_excel('trips.xlsx', header=0)  # header=0 means the first row is the header
df_trips.sample(2)

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition
2113,77403 (М947ОК67) Renault Logan,18.02.2025 00:57:02,"55,751951°, 37,81484°",17,+
2997,77403 (М947ОК67) Renault Logan,18.02.2025 15:33:41,"55,766893°, 37,830915°",0,-


In [72]:
# Split the 'Coordinates' column into 'Longitude' and 'Latitude'
df_trips['Сoordinates'] = df_trips['Сoordinates'].str.replace('°', '').str.replace(',', '.')
df_trips[['Longitude', 'Latitude']] = df_trips['Сoordinates'].str.split('. ', expand=True)
# df_trips.drop(['Сoordinates'], axis=1, inplace=True)

# Convert 'Day_time' to datetime objects
df_trips['Day_time'] = pd.to_datetime(df_trips['Day_time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')

df_trips['Ignition'] = df_trips['Ignition'].str.replace('-', '0').str.replace('+', '1')

df_trips = df_trips.sort_values(['Car_ID', 'Day_time'])  # Сортируем по времени

# Преобразование поездок в геометрию
df_trips['geometry'] = df_trips.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
df_trips = gpd.GeoDataFrame(df_trips, geometry='geometry', crs="EPSG:4326")

df_trips.sample(2)

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Longitude,Latitude,geometry
344,77403 (М947ОК67) Renault Logan,2025-02-17 07:47:09,55.77863. 37.827593,10,1,55.77863,37.827593,POINT (55.77863 37.82759)
3532,77403 (М947ОК67) Renault Logan,2025-02-18 22:47:14,55.767211. 37.82354,0,1,55.767211,37.82354,POINT (55.76721 37.82354)


In [73]:
# Загрузка данных о Выделенных полосах
gdf_bus_lanes = gpd.read_file("bus-lanes.geojson")
gdf_bus_lanes = gdf_bus_lanes.set_crs("EPSG:4326")

In [34]:
# # Создаем траектории машин
# def create_trajectories(df):
#     trajs = []
#     for car_id, group in df.groupby('Car_ID'):
#         points = [Point(lon, lat) for lon, lat in zip(group['Longitude'], group['Latitude'])]
#         trajs.append({'Car_ID': car_id, "geometry": LineString(points)})
#     return gpd.GeoDataFrame(trajs, crs="EPSG:4326")

# gdf_trips = create_trajectories(df_trips)

In [74]:
# Добавляем buffer(0.0003) к выделенным полосам - примерно 30 метров
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=3857)  # Web Mercator (метры)
gdf_bus_lanes["geometry"] = gdf_bus_lanes["geometry"].buffer(30)  # 30 метров (0.0003 градусов ~ 30 м)
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=4326)  # Возвращаем в исходную систему

# Определяем пересечения с учетом buffer
df_trips["on_bus_lane"] = df_trips.intersects(gdf_bus_lanes["geometry"].union_all())

In [75]:
# Анализ скорости на участках
df_trips["next_time"] = df_trips.groupby('Car_ID')['Day_time'].shift(-1)
df_trips["next_lon"] = df_trips.groupby('Car_ID')['Longitude'].shift(-1)
df_trips["next_lat"] = df_trips.groupby('Car_ID')['Latitude'].shift(-1)

# Вычисляем расстояние между последовательными точками в км
df_trips['distance'] = df_trips.apply(
    lambda row: geodesic((row['Latitude'], row['Longitude']), (row["next_lat"], row["next_lon"])).meters / 1000
    if pd.notnull(row["next_lat"]) else None,
    axis=1
)

# Вычисляем время между последовательными точками, часов
df_trips["time"] = df_trips.apply(
    lambda row: (row["next_time"] - row['Day_time']).total_seconds() / 3600 if pd.notnull(row["next_time"]) else None,
    axis=1
)

# Вычисляем сколрость между последовательными точками км/ч
df_trips["speed"] = df_trips.apply(
    lambda row: row["distance"] / row["time"] if pd.notnull(row["distance"]) and row["time"] not in [None, 0] else None,
    axis=1
)

In [77]:
df_trips.iloc[236:239]

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Longitude,Latitude,geometry,on_bus_lane,next_time,next_lon,next_lat,distance,time,speed
236,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:27,55.767596. 37.83425,15,1,55.767596,37.83425,POINT (55.7676 37.83425),False,2025-02-17 06:53:31,55.76751,37.834183,0.010612,0.001111,9.550852
237,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:31,55.76751. 37.834183,7,1,55.76751,37.834183,POINT (55.76751 37.83418),False,2025-02-17 06:53:32,55.767491,37.834136,0.005478,0.000278,19.721727
238,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:32,55.767491. 37.834136,14,1,55.767491,37.834136,POINT (55.76749 37.83414),False,2025-02-17 06:53:33,55.767483,37.834058,0.008686,0.000278,31.269875


In [79]:
# Оценка скорости на выделенных полосах
avg_speed_bus_lane = 50

# Оценка выигрыша во времени
df_trips['predicted_time'] = df_trips.apply(
    lambda row: min(row['time'], row['distance'] / avg_speed_bus_lane) if row["on_bus_lane"] else row['time'],  # Added condition to handle zero speed
    axis=1
)

In [81]:
df_trips.iloc[236:239]

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Longitude,Latitude,geometry,on_bus_lane,next_time,next_lon,next_lat,distance,time,speed,predicted_time
236,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:27,55.767596. 37.83425,15,1,55.767596,37.83425,POINT (55.7676 37.83425),False,2025-02-17 06:53:31,55.76751,37.834183,0.010612,0.001111,9.550852,0.001111
237,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:31,55.76751. 37.834183,7,1,55.76751,37.834183,POINT (55.76751 37.83418),False,2025-02-17 06:53:32,55.767491,37.834136,0.005478,0.000278,19.721727,0.000278
238,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:32,55.767491. 37.834136,14,1,55.767491,37.834136,POINT (55.76749 37.83414),False,2025-02-17 06:53:33,55.767483,37.834058,0.008686,0.000278,31.269875,0.000278


In [ ]:
# время в движении, часов
df_trips['time_on_bus_lane'] = df_trips.loc[df_trips['speed'] > 0, 'time'].sum()


df_trips["time_saved"] = df_trips["time"] - df_trips["predicted_time"]
# Итоговый выигрыш во времени
total_time_saved = df_trips["time_saved"].sum()

print(f"Общая экономия времени: {total_time_saved / 3600:.2f} часов")

# df_trips["time"] в ячейках где df_trips["speed"] >0


